<a href="https://colab.research.google.com/github/DariusBotusanu/RoFinBert/blob/main/fine_tuning_colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! pip install transformers datasets

ERROR: Could not find a version that satisfies the requirement ffspec (from versions: none)
ERROR: No matching distribution found for ffspec


In [12]:
from datasets import load_dataset

# If the dataset is gated/private, make sure you have run huggingface-cli login
dataset = load_dataset("dariusbotusanu/custom_ro_financial_phrasebank")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]